# Zgrubny opis metod i rozwiązań

1. Coś tam z merkatorem i reprezentacją pomiarów z gps na mapce:
- nasze dane wydają się ładnie rzutować na miejsca, w których powinny się znaleźć
- czy tak samo jest z surowego odczytu z GPS? Czy my mamy surowy odczyt z GPS? Nie wiem
2. Pozyskanie tras na podstawie współrzędnych geograficznych:
- sposób pierwszy: obliczanie odległości w linii prostej z twierdzenia pitagorasa ( można uzyć jako dolne ograniczenie)
- sposób drugi: dystans geodezyjny (dystans z uwzględnieniem krzywizny Ziemi, poprawny sposób do obliczania odległości w linii prostej, ale nam się nie przyda, biblioteka geopy)
- sposób trzeci: wyznaczenie tras przy użyciu API dowolnego silnika trasującego (przykłady to OSRM, Valhalla, GraphHopper, Google, korzystamy z tego pierwszego). Działa to dobrze i zawiera wiele dodatkowych informacji, my korzytamy jedynie z dystansu określonej trasy (routingpy, działa średnio, API client Jana Pomykacza, działa zajebiście xd).
- czego nie sposób odtworzyć: bez dodatkowych informacji nie jesteśmy w stanie określić natężenia ruchu, blokad, remontów, zmian tras itp. Są to trudno dostępne informacje, gdyż w trakcie dowozu jedzenia kurier zazwyczaj nie ma niewiadomo ilu czujników, tudzież aplikacji szpiegującej na telefonie.
3. Wizualizacje:
- Plotowanie bezpośrednie: folium: umożliwia tworzenie przybliżalnych map w html. Wiele tras powoduje spadek responsywności mapy, gdyż są to grafiki wektorowe. Umożliwia tworzenie heatmap (choropleth).
- Wygenerowanie grafiki rastrowej i nałożenie na mapę: (GDAL + Folium, albo inna libka do mapek): routy generują się do bitmap, które poźniej nakładamy na mapę
- Wygenerowanie heatmapy: (folium + ????): podzielenie mapy na kwadraty NxN metrów, jeśli route przechodzi przez kwadrat to zwiększ liczbę, wyplotuj kwadraty jako heatmapę.


In [5]:
import folium
import requests

# Define start and end points
start_point = (22.745049, 75.892471)  # jakieś gunwo w berlinie
end_point = (22.765049 , 75.912471)     # jakieś gunwo w berlinie kawołek dalej

# points = (((75.892471, 22.745049)),((75.912471, 22.765049)))
# Create a map centered around the start point
mymap = folium.Map(location=start_point, zoom_start=6)
# for start_point, end_point in points:
    # Define the OSRM API URL
osrm_url = f'http://router.project-osrm.org/route/v1/driving/{start_point[1]},{start_point[0]};{end_point[1]},{end_point[0]}?overview=false&geometries=geojson&steps=true'

# Make a request to the OSRM API
response = requests.get(osrm_url)
data = response.json()
# steps = data['routes'][0]['legs'][0]['steps']

# Extract route coordinates from the response
route_coordinates = [[(point[1], point[0]) for point in data['routes'][0]['legs'][0]['steps'][i]['geometry']['coordinates']] for i in range(len(data['routes'][0]['legs'][0]['steps']))]

# Add a polyline to represent the route
folium.PolyLine(locations=route_coordinates, color='blue').add_to(mymap)

# Add markers for the start and end points
folium.Marker(location=start_point, icon=folium.Icon(color='green')).add_to(mymap)
folium.Marker(location=end_point, icon=folium.Icon(color='red')).add_to(mymap)

# Save the map to an HTML file
mymap.save("nearest_map.html")


In [2]:
import folium
import pandas as pd
import ast

df_routes = pd.read_csv('routes/routes2.csv', delimiter=',')

routes = df_routes['Route-coordinates']
center_eendia_point = (21.9149, 78.0281)

# Create a map centered around the start point
mymap = folium.Map(location=center_eendia_point, zoom_start=6, prefer_canvas=True)
for str_route in routes:
    # Define the OSRM API URL
    route_coordinates = ast.literal_eval(str_route)
    # Extract route coordinates from the response
    # Add a polyline to represent the route
    # folium.PolyLine(locations=route_coordinates, color='rgba(0, 0, 255, 0.25)', weight=4).add_to(mymap)

    for coord in route_coordinates:
        for point in coord:
            folium.Circle(location=point, radius=1, color='rgba(0, 0, 255, 0.25)', fill=True,  weight=4).add_to(mymap)

    # Add markers for the start and end points
    # folium.Marker(location=start_point, icon=folium.Icon(color='green')).add_to(mymap)
    # folium.Marker(location=end_point, icon=folium.Icon(color='red')).add_to(mymap)

# Save the map to an HTML file
mymap.save("whole_eeendia.html")


In [1]:
import folium
import json
import requests

# URL for a simple GeoJSON file for US states (you might need to find or host a suitable GeoJSON file for real-world applications)
url = (
    "https://raw.githubusercontent.com/python-visualization/folium/main/examples/data"
)
state_geo = f"{url}/us-states.json"
geo_json_data = json.loads(requests.get(state_geo).text)

# Simulated data: number of points per state (example)
# In practice, this should be the result of your data processing
state_data = {
    'Alabama': 10, 'Alaska': 5, 'Arizona': 15, 'Arkansas': 7, 'California': 30,
    'Colorado': 20, 'Connecticut': 12, 'Delaware': 5, 'Florida': 25, 'Georgia': 17,
    # Add data for all states or ensure your GeoJSON keys match your data keys exactly
}

# Initialize a map centered on the USA
mymap = folium.Map(location=[37.8, -96], zoom_start=4)

# Create a choropleth map
folium.Choropleth(
    geo_data=geo_json_data,
    name='choropleth',
    data=state_data,
    columns=['State', 'Density'],
    key_on='feature.properties.name',
    fill_color='YlOrRd',  # Color scheme
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Sample Points Per State'
).add_to(mymap)

folium.LayerControl().add_to(mymap)

# Save to HTML
mymap.save('us_states_heatmap.html')


In [8]:
import folium
import json
import requests

# URL for a GeoJSON file for Indian states
url = "https://raw.githubusercontent.com/Subhash9325/GeoJson-Data-of-Indian-States/master/Indian_States"
state_geo = url
geo_json_data = json.loads(requests.get(state_geo).text)

# Simulated data: number of points per state (example)
# In practice, this should be the result of your data processing
state_data = {
    'Andhra Pradesh': 10, 'Arunachal Pradesh': 5, 'Assam': 15, 'Bihar': 7, 'Chhattisgarh': 30,
    'Goa': 20, 'Gujarat': 12, 'Haryana': 5, 'Himachal Pradesh': 25, 'Jharkhand': 17,
    'Karnataka': 10, 'Kerala': 5, 'Madhya Pradesh': 15, 'Maharashtra': 7, 'Manipur': 30,
    'Meghalaya': 20, 'Mizoram': 12, 'Nagaland': 5, 'Odisha': 25, 'Punjab': 17,
    'Rajasthan': 10, 'Sikkim': 5, 'Tamil Nadu': 15, 'Telangana': 7, 'Tripura': 30,
    'Uttar Pradesh': 20, 'Uttarakhand': 12, 'West Bengal': 5, 'Andaman and Nicobar Islands': 25,
    'Chandigarh': 17, 'Dadra and Nagar Haveli and Daman and Diu': 10, 'Lakshadweep': 5,
    'Delhi': 15, 'Puducherry': 7,
    # Add data for all states or ensure your GeoJSON keys match your data keys exactly
}

# Initialize a map centered on India
mymap = folium.Map(location=[20.5937, 78.9629], zoom_start=4)

# Create a choropleth map
folium.Choropleth(
    geo_data=geo_json_data,
    name='choropleth',
    data=state_data,
    columns=['State', 'Density'],
    key_on='feature.properties.NAME_1',
    fill_color='YlOrRd',  # Color scheme
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Sample Points Per State (India)'
).add_to(mymap)

folium.LayerControl().add_to(mymap)

# Save to HTML
mymap.save('india_states_heatmap.html')


In [10]:
import folium
import pandas as pd
import ast
from geojson import Polygon, Feature, FeatureCollection

df_routes = pd.read_csv('routes/routes2.csv', delimiter=',')

routes = df_routes['Route-coordinates']
center_eendia_point = (21.9149, 78.0281)
# routes = routes.head(20)
polygon_collection = []
for str_route in routes:
    route = ast.literal_eval(str_route)
    polygon = Polygon(route)
    polygon_collection.append(Feature(geometry=polygon))
polygons = FeatureCollection(polygon_collection)
mymap = folium.Map(location=center_eendia_point, zoom_start=6, prefer_canvas=True)
geojson_data = folium.GeoJson(polygons, style_function=lambda feature:{"color":"blue"}).add_to(mymap)
# mymap.fit_bounds(geojson_data.get_bounds())
mymap.save("goejson_test.html")

: 